In [20]:
from functools import reduce

import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hipotesis = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr

    '''
        probabilidad(clase) =   frecuencia de aparcicin or columna/ conteo clase
    '''
    def calcular_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Valores a priori : ", self.priori)

    '''
        Calculo de las probabilidades individuales
        P(outcome|evidencia) =   P(verosimilitud de la evidencia) x Prior prob de outcome/P(Evidencia)

    '''
    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    '''
        Aquí se calcula la Probabilidad de Evidencia y se multiplica todas las probabilidades individuales a priori
        (Outcome|Multiple Evidence) = P(Evidence1|Outcome) x P(Evidence2|outcome) x ... x P(EvidenceN|outcome) x P(Outcome)
        escalado por P(Multiple Evidence)
    '''
    def calcular_probabilidad_condicional(self, hipotesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hipotesis:
                self.cp[i].update({ hipotesis[j]: self.get_cp(j, hipotesis[j], i)})
        print ("\nProbabilidades condicionales calculadas: \n")
        pprint.pprint(self.cp)

    def clasificar(self):
        print ("Resultado: ")
        for i in self.cp:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cp[i].values())*self.priori[i])

if __name__ == "__main__":
    c = Classifier(filename="new_dataset.csv", class_attr="Play" )
    c.calcular_priori()
    c.hipotesis =  {"Outlook":'Overcast'}

    c.calcular_probabilidad_condicional(c.hipotesis)
    c.clasificar()

Valores a priori :  {'no': 0.35714285714285715, 'yes': 0.6428571428571429}

Probabilidades condicionales calculadas: 

{'no': {'Overcast': 0.2}, 'yes': {'Overcast': 0.5555555555555556}}
Resultado: 
no  ==>  0.07142857142857144
yes  ==>  0.3571428571428572


In [19]:
from functools import reduce

import pandas as pd
import pprint

class Classifier():
    data = None
    class_attr = None
    priori = {}
    cp = {}
    hypothesis = None


    def __init__(self,filename=None, class_attr=None ):
        self.data = pd.read_csv(filename, sep=',', header =(0))
        self.class_attr = class_attr

    '''
        probability(class) =    How many  times it appears in cloumn
                             __________________________________________
                                  count of all class attribute
    '''
    def calculate_priori(self):
        class_values = list(set(self.data[self.class_attr]))
        class_data =  list(self.data[self.class_attr])
        for i in class_values:
            self.priori[i]  = class_data.count(i)/float(len(class_data))
        print ("Priori Values: ", self.priori)

    '''
        Here we calculate the individual probabilites 
        P(outcome|evidence) =   P(Likelihood of Evidence) x Prior prob of outcome
                               ___________________________________________
                                                    P(Evidence)
    '''
    def get_cp(self, attr, attr_type, class_value):
        data_attr = list(self.data[attr])
        class_data = list(self.data[self.class_attr])
        total =1
        for i in range(0, len(data_attr)):
            if class_data[i] == class_value and data_attr[i] == attr_type:
                total+=1
        return total/float(class_data.count(class_value))

    '''
        Here we calculate Likelihood of Evidence and multiple all individual probabilities with priori
        (Outcome|Multiple Evidence) = P(Evidence1|Outcome) x P(Evidence2|outcome) x ... x P(EvidenceN|outcome) x P(Outcome)
        scaled by P(Multiple Evidence)
    '''
    def calculate_conditional_probabilities(self, hypothesis):
        for i in self.priori:
            self.cp[i] = {}
            for j in hypothesis:
                self.cp[i].update({ hypothesis[j]: self.get_cp(j, hypothesis[j], i)})
        print ("\nCalculated Conditional Probabilities: \n")
        pprint.pprint(self.cp)

    def classify(self):
        print ("Result: ")
        for i in self.cp:
            print (i, " ==> ", reduce(lambda x, y: x*y, self.cp[i].values())*self.priori[i])

if __name__ == "__main__":
    c = Classifier(filename="new_dataset.csv", class_attr="Play" )
    c.calculate_priori()
    c.hypothesis = {"Outlook":'Overcast'}

    c.calculate_conditional_probabilities(c.hypothesis)
    c.classify()

Priori Values:  {'no': 0.35714285714285715, 'yes': 0.6428571428571429}

Calculated Conditional Probabilities: 

{'no': {'Overcast': 0.2}, 'yes': {'Overcast': 0.5555555555555556}}
Result: 
no  ==>  0.07142857142857144
yes  ==>  0.3571428571428572
